# Importamos las librerias

In [ ]:
import numpy as np
import os
import math
from itertools import combinations,product

# Cargamos el dataset

In [2]:
data=np.load('spotify.npy',allow_pickle=True) 


# Algoritmo Apriori ami manera

In [38]:
Playlists = []
for i in data.item():
    Playlists.append(data.item()[i])
     
def difConjuntos(X,Y):
    X=list(X)
    Y=list(Y)
    primera=''
    segunda=''  
    for i in Y:
        if i not in X:
            segunda+=i
            segunda+=','
        else:
            primera+=i
            primera+=','

    primera=primera[:-1]
    segunda=segunda[:-1]
    if(primera=='' or segunda==''):
        return 0,0
    else:
        return primera,segunda


def BuscandoMatch(Lista,Valor):
    
    Itemset = {}
    for i in range (len(Lista)):
        m = 0 
        ListaIndices = [] 
        for j in Lista[i]:
            for h in Valor:
                if(j==h):
                    m += 1
                    
        if(m==len(Valor)):
            if str(Valor) not in Itemset:
                Itemset.setdefault(str(Valor), [])
            else:
                Itemset.setdefault(str(Valor), []).append(i)
    return(Itemset)


#Aqui usare una lista para guardar las playlists
def get_frequent_itemsets(Playlists, min_support):
    Canciones = {}

    for ListaCancion in Playlists:
        for Cancion in ListaCancion:
            if Cancion not in Canciones:
                Canciones[Cancion] = 1
            else:
                Canciones[Cancion] += 1
    canciones= {}
    for key,value in Canciones.items():
        valor=value/10000
        if(valor > min_support):
            canciones[key] =valor
    return canciones          

    
def generate_association_rules(Playlist,frequent_itemsets, confidence = 0.1, lift =1.2):
    #Guardando La lista de frequent_items en ListaCaniones
    ListaCanciones=[]
    for key,value in frequent_itemsets.items():
        ListaCanciones.append(key)
    # Generando Combinaciones de n
    combinaciones = []
    #LAs combinaciones empizan de 2
    numeroCombinaciones = 2
    Cumple=5
    #Diccionario para las Reglas
    Reglas={}
    #Diccionario para los valores de support
    support = {}
    support.update(frequent_itemsets)
    #Diccionario para LAs reglas y sus resultados
    ResultadoFinal={}
    ResultadoFinal.update(frequent_itemsets)
    #Hacemos la operacion hasta que dejen de existir tales combnaciones
    while(Cumple):
        comb = combinations(ListaCanciones, numeroCombinaciones)
        contador = 0
        #Generamos las combinaciones de reglas
        for i in comb:
            ListaCreadaDentro = 0
            #Hacemos las diferencias entrelas cobinaciones 
            #Y los items de las canciones
           
            i = list(i)
            for j in ListaCanciones:
                
                x,y = difConjuntos([j],i)
                if(x==0 and y==0):
                    #Estas son combinaciones que no exiten
                    pass
                else:
                    
                    valors = x + ',' + y
                    
                    valor = valors.rsplit(',')
                    #Hacemos el conteo de cuantas listas contienen a la
                    #sublista a analizar
                    ListaFiltrada = BuscandoMatch(Playlist,valor)
                    nume = 0
                    #Recuperamos el contador de las listas que contienen a la
                    #sublista a analizar
                    for key,value in ListaFiltrada.items():
                        nume = len(ListaFiltrada.setdefault(key, []))
                    if(x not in support):
                        pass
                    else:
                        #Calculamos el valor de la confianza
                        confianza = nume/(support[x]*10000)
                        #Calculamos el valor del suport
                        suport = nume/10000
                        support[valors] = suport
                        #CAlculamos el denominador para el lift
                        denominador = 1
                        for n in valor:
                            if(n in support):
                                denominador = denominador * support[n] * 10000
                        #Hacemos la division del lift
                        lift = suport/denominador

                        if(confianza >= confidence ):
                            #Verificamos si el valor de la confianza es mayor que 
                            #el limite dado
                            ListaCreadaDentro += 1
                            valorn = x + '=>' + y
                            Reglas[valorn] = 0
                            #Guardamos los valores
                            Lista = [suport,confianza,lift]
                            ResultadoFinal[valors] = Lista
                        

                
        numeroCombinaciones+=1
        Cumple-=1
    
    return ResultadoFinal,Reglas    
    
#Lista de Canciones frecuentes con pesos
frequent_itemsets=get_frequent_itemsets(Playlists,0.04)
for key,value in frequent_itemsets.items():
    print(key+"  =>  "+str(value))
#print(frequent_itemsets["Jumpin"])

combi,Reglas=generate_association_rules(Playlists,frequent_itemsets)
for key,value in Reglas.items():
    print(key)
print('REGLA-----------support---confidence---value')
for key,value in combi.items():
    print(key+"  =>  "+str(value))


Closer  =>  0.0761
Home  =>  0.0489
One Dance  =>  0.0433
Congratulations  =>  0.041
Let Me Love You  =>  0.0403
Roses  =>  0.0436
Ride  =>  0.0429
HUMBLE.  =>  0.047
Broccoli (feat. Lil Yachty)  =>  0.0402
Closer=>Home
Home=>Closer
Closer=>One Dance
One Dance=>Closer
Closer=>Congratulations
Congratulations=>Closer
Closer=>Let Me Love You
Let Me Love You=>Closer
Closer=>Roses
Roses=>Closer
Closer=>Ride
Ride=>Closer
Closer=>HUMBLE.
HUMBLE.=>Closer
Closer=>Broccoli (feat. Lil Yachty)
Broccoli (feat. Lil Yachty)=>Closer
Home=>One Dance
One Dance=>Home
Home=>Let Me Love You
Let Me Love You=>Home
Home=>Roses
Roses=>Home
Home=>Ride
Ride=>Home
One Dance=>Congratulations
Congratulations=>One Dance
One Dance=>Let Me Love You
Let Me Love You=>One Dance
One Dance=>Roses
Roses=>One Dance
One Dance=>Ride
Ride=>One Dance
One Dance=>HUMBLE.
HUMBLE.=>One Dance
One Dance=>Broccoli (feat. Lil Yachty)
Broccoli (feat. Lil Yachty)=>One Dance
Congratulations=>Let Me Love You
Let Me Love You=>Congratulations

Segun a estas 4 regla se puede deducir, que muy aparte de estar en tendencia, tambien guardan relacion entre ellas ya se por las letras o por el ritmo que tienen en comun:
Home,Closer  =>  [0.0105, 0.2147239263803681, 2.8216021863386086e-08]
Ride,Closer,Let Me Love You  =>  [0.0057, 0.13286713286713286, 4.332393150814778e-11]
Ride,Closer  =>  [0.0148, 0.34498834498834496, 4.533355387494678e-08]
Broccoli (feat. Lil Yachty),Congratulations,HUMBLE.  =>  [0.0076, 0.1890547263681592, 9.810831674528241e-11]
El valor del suport solo nos indicara que tanto de porcentaje tiene esta relacion respecto a toda la data
El valor de confidence nos dara la relacion de que tanto frecuente son sus valores respecto a la relacion que contiene X. que en este caso se relacionara y aumentara segun a cuantas veces aparezca el X dentro de la Playlist
El valor de lift sera para darle importancia a esa relacion en analisis